# Example code of GPT API
Colab 환경에서 GPT 모델 API를 사용해 손글씨를 인식해보는 예제입니다.

## 패키지 및 예제 데이터 다운로드하기
예제를 실행시키기 위해 python package들을 설치합니다. Colab에서 실행하지 않는 경우 이 셀은 실행하지 않습니다.

In [ ]:
!wget https://raw.githubusercontent.com/mentor1023/dl_apps/main/nlp/requirements-colab.txt
!pip install -r requirements-colab.txt

## 패키지 불러오기

In [ ]:
import json
from openai import OpenAI

## GPT API 사용을 위한 환경 설정하기
OpenAI Platform의 계정에 연결된 API 키를 사용합니다. 그리고 어떤 모델을 사용할 지 설정합니다. 이 예제에서는 `gpt-3.5-turbo`를 사용합니다.

In [ ]:
# openai API 키 인증
client = OpenAI(api_key="<OPENAI_API_KEY>")

In [ ]:
# 모델 - GPT 3.5 Turbo 선택
model = "gpt-3.5-turbo"

## 프롬프트 작성하여 GPT API에 요청할 내용 만들기

In [ ]:
# 프롬프트 작성하기
prompt = "Describe ChatGPT in 40 words or less."

In [ ]:
# 메시지 설정하기
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]

### GPT API 요청하기

In [ ]:
response = client.chat.completions.create(model=model, messages=messages)

In [ ]:
print(response.model_dump_json(indent=2))

In [ ]:
response.choices[0].message.content

### 함수 호출 기능(Function calling) 사용하기

In [ ]:
def get_current_weather(location: str):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "20",
        "unit": "Celsius",
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
function_signature = {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA",
            },
        },
        "required": ["location"],
    },
}

In [ ]:
prompt = "What's the weather like in Seoul?"
messages = [{"role": "user", "content": prompt}]

In [ ]:
response = client.chat.completions.create(
    model=model,
    messages=messages,
    functions=[function_signature],
)
response_message = response.choices[0].message

In [ ]:
print(response_message.model_dump_json(indent=2))

In [ ]:
function_name = response_message.function_call.name
function_args = json.loads(response_message.function_call.arguments)
function_response = get_current_weather(location=function_args["location"])

In [ ]:
function_response

In [ ]:
messages.append(response_message)
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": function_response,
    }
)

In [ ]:
messages

In [ ]:
second_response = client.chat.completions.create(model=model, messages=messages)

In [ ]:
second_response.choices[0].message.content